In [ ]:
### Imports
from IPython.display import display
from ipywidgets import interactive
import ipywidgets as widgets
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Grundlegende Struktur von künstlichen neuronalen Netzwerken

**[Künstliche neuronale Netze](https://de.wikipedia.org/wiki/K%C3%BCnstliches_neuronales_Netz)** gehen in ihrer konzeptionellen Grundlagen auf die Arbeit von [Warren McCulloch](https://de.wikipedia.org/wiki/Warren_McCulloch) und [Walter Pitts](https://de.wikipedia.org/wiki/Walter_Pitts) zurück, die bereits $1943$ in Analogie zu Neuronen verknüpfte Netzwerke zur räumlichen Mustererkennung vorschlugen. $1958$ gelang [Frank Rosenblatt et al.](https://de.wikipedia.org/wiki/Frank_Rosenblatt) in Form des **[Perzeptrons](https://de.wikipedia.org/wiki/Perzeptron)** die erste praktische Umsetzung eines neuronalen Netzwerks. $1969$ führte die Kritik von [Marvin Minsky](https://de.wikipedia.org/wiki/Marvin_Minsky) an der Unfähigkeit, mit einfachen Perzeptrons nicht linear separierbare Probleme (wie zum Beispiel beim **[XOR-Problem](https://de.wikipedia.org/wiki/Exklusiv-Oder-Gatter)**)  zu lösen, zu einem zeitweisen Rückgang des Forschungsinteresses (dem sogenannten KI-Winter). Dies änderte sich in den $1980$er Jahren, als durch verschiedene Fortschritte in der KI-Forschung wie zum Beispiel der Methode der **[Backpropagation](https://de.wikipedia.org/wiki/Backpropagation)** gezeigt werden konnte, dass mehrschichtige Perzeptrons auch in der Lage sind, nicht linear separierbare Probleme zu bewältigen. 

Wir haben an anderer Stelle im Einzelnen über die verschiedenen Arten des maschinellen Lernens - das **[unüberwachte Lernen](https://de.wikipedia.org/wiki/Un%C3%BCberwachtes_Lernen)**, **[überwachte Lernen](https://de.wikipedia.org/wiki/%C3%9Cberwachtes_Lernen)** und **[bestärkende Lernen](https://de.wikipedia.org/wiki/Best%C3%A4rkendes_Lernen)** - gesprochen. Wir haben verschiedene Machine-Learning-Algorithmen vorgestellt, die bei den unterschiedlichen Lernarten zum Einsatz kommen. Neuronale Netze zeichnen sich insbesondere dadurch aus, dass sie bei entsprechender Vorbereitung in allen drei Arten des Lernens erfolgreich eingesetzt werden können. Diese universelle Verwendbarkeit erklärt auch den vermehrten Einsatz von künstlichen neuronalen Netzwerken in unterschiedlichsten Bereichen.

In diesem Kapitel werden wir uns mit den Stärken und Schwächen von **neuronalen Netze** sowie deren Einsatzmöglichkeiten und der zugrunde liegenden mathematischen Formulierung beschäftigen.

## Aufbau der Nervenzelle

Sehen wir uns zuerst das Vorbild für neuronale Netze an: die Nervenzelle.

<img src="./images/realistische-neuronenanatomie_1284-68077.avif" alt="drawing" width="80%"/>

Eine Nervenzelle besteht, wie in der oberen Abbildung dargestellt, aus **Dendriten**, **Soma** und **Axon**. Dabei nehmen die **Dendriten** Botenstoffe, die sogenannten Neurotransmitter auf wenn diese von angeregten benachbarten Nervenzellen ausgeschüttet werden. Die Verbindungen zwischen den Dendriten und dem Axon der vorhergehenden Zelle bezeichnet man als **Synapse**. Das Neuron besitzt ein Membranpotential das ein Auslösen der Weitergabe eines Nervenreizes zuerst einmal unterdrückt. Erst beim Überschreiten eines gewissen **Schwellenwerts der Anregung** wird ein **Aktionspotential** ausgelöst und ein Nervenreiz über das **Axon** an die nächste Zelle weitergegeben. 

Dabei gilt bei der Anregung des Neurons das sogenannte **[Alles-oder-nichts-Gesetz](https://de.wikipedia.org/wiki/Alles-oder-nichts-Gesetz)**, dass aussagt das entweder ein Reiz vollständig ausgelöst wird oder gar nicht.

Dies lässt sich mathematisch mit der **Heaviside-Sprungfunktion** ausdrücken, die definiert ist als:

$$H(x) \begin{cases} x \lt 0 \cdots 0 \\ x \ge 0 \cdots 1 \end{cases} $$

In [ ]:
# Heaviside-Sprungfunktion
def heaviside(x):
    return 0.5 * (np.sign(x) + 1)


# Erzeuge Werte für x
x = np.linspace(-5, 5, 1000)

# Berechne die Werte der Heaviside-Sprungfunktion
y = heaviside(x)

# Plot
plt.figure(dpi=600, figsize=(6, 3))
plt.rcParams.update({'font.size': 6})
plt.tight_layout()
plt.plot(x, y, linewidth = 2.5, label="Heaviside-Sprungfunktion")
plt.xlabel("x")
plt.ylabel("H(x)")
plt.title("Heaviside-Sprungfunktion")
plt.ylim([-0.5,1.5])
plt.grid(True)
plt.legend()
plt.show()

Wir haben also biologisch gesehen einen **variablen Reiz**, einen **Schwellenwert** der überschritten werden muss und eine **Aktivierungsfunktion** zur Auslösung des Reizes.

## Einfaches Perzeptron

Betrachten wir als nächstes den einfachsten Aufbau eines neuronalen Netzwerks, des ursprünglich von Rosenblatt vorgeschlagenen **Pezeptrons**. Dieses ist aus zwei Eingängen, dem Neuron selbst und einem Ausgang aufgebaut, wie dies in der folgenden Abbildung dargestellt wird.

<img src="./images/perceptron.png" alt="drawing" width="80%"/>

Die Struktur von **künstlichen neuronalen Netzwerken** folgt im Grundprinzip dem Aufbau eines **biologischen Nervensystems**. In neuronalen Netzen entsprechen Neuronen den **[künstlichen Neuronen](https://de.wikipedia.org/wiki/K%C3%BCnstliches_Neuron)** oder Knoten im Netzwerk. Diese Neuronen sind die grundlegenden Verarbeitungseinheiten. 

Künstliche neuronale Netzwerke bestehen aus einzelnen Neuronen die in sogenannten Schichten (*engl. Layers*) angeordnet sind. Dabei besteht die erste Schicht oder Eingabeschicht (*engl. Input Layer*) aus den Eingabewerten, gefolgt von weiteren Schichten von Neuronen, den sogenannten versteckten Schichten (*engl. Hidden Layers*) und schließlich einer Ausgabeschicht (*engl. Output Layer*).

In Analogie zu biologischen Neuronen, die durch Anregung über einen gewissen Grenzwert aktiviert werden, Reize weiterzuleiten, wird bei neuronalen Netzwerken die gewichtete Summe der Inputs an verbundene Neuronen weitergegeben. Bezogen auf ein Neuron ergibt sich:

$$ \text{Input} = \sum_{i=1}^N x_i w_i $$

Dabei sind die $x_i$ die einzelnen Eingabewerte und die $w_i$ die jeweiligen Gewichtungen der $N$ Inputs.

Im Äquivalent zu der biologischen Schwelle, ab der ein Neuron aktiviert wird, um ein Signal weiterzuleiten, können wir einen Schwellenwert $b$ hinzufügen, den sogenannten Bias.

$$ \text{Input} = \sum_{i=1}^N x_i w_i + b$$

Auf die auf diese Weise berechnete Ausgabe müssen wir noch normalerweise eine geeignete Aktivierungsfunktionen anwenden.

$$f_{\text{Aktiv}}(\text{Input}) = f_{\text{Aktiv}}(\sum_{i=1}^N x_i w_i + b) = \text{Output}$$

Die **Aktivierungsfunktion** bildet die gewichtete Summe der Eingabewerte auf einen bestimmten Wertebereich ab und dient dazu, die Ausgabe eines Neurons oder einer Schicht zu steuern. Sie entscheidet, ob und in welchem Maße ein Neuron aktiviert wird und welche Informationen an die nächsten Schichten weitergegeben werden. Im Weiterern ermöglichen geeignete Aktivierungsfunktionen, auch nicht lineare Zusammenhänge zu beschreiben. Auf die genaue Form von verschiedenen Aktivierungsfunktionen werden wir später zurückkommen und gehen für den Moment von linear weitergeleiteten Werten aus.

Also mathematisch betrachtet 

$$f_{\text{Aktiv}}(\text{Input}) = \text{Input}$$ 

und damit gilt

$$ \text{Input} = \sum_{i=1}^N x_i w_i + b = \text{Output}$$

In [ ]:
### Abildung Gewichte und Bias
plt.rcParams.update({'font.size': 10})
# Werte für Gewicht und Bias
weights = [3, 2, 1]  # Gewichtswerte für den ersten Plot
bias_values = [0]  # Bias für den ersten Plot (Bias = 0)
weight_constant = 1  # Konstanter Wert für Gewicht im zweiten Plot
bias_values_2 = [1, 0, -1]  # Verschiedene Bias-Werte für den zweiten Plot

# Erstellen zweier Subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6), dpi = 600)

# Plot für die Gewichtswerte mit Bias = 0
for weight in weights:
    ax1.plot([-2, -1, 0, 1, 2], [weight * x + 0 for x in [-2, -1, 0, 1, 2]], label=f"Weight = {weight}")
ax1.set_title("Verschiedene Gewichte bei konstantem Bias")
ax1.set_xlabel("x")
ax1.set_ylabel("y = weight * x + bias")
ax1.legend()
ax1.grid(True)
ax1.set_xticks(np.arange(-2,3,1))
ax1.set_yticks(np.arange(-6,7,1))
ax1.axhline(0, 2, color='black',linewidth=1)
ax1.axvline(0, 6, color='black',linewidth=1)
ax1.annotate('', xy=(2, 0), xytext=(0, 0), arrowprops=dict(facecolor='black', edgecolor='black', arrowstyle='->', lw=1))
ax1.annotate('', xy=(0, 6), xytext=(0, 0), arrowprops=dict(facecolor='black', edgecolor='black', arrowstyle='->', lw=1))


# Plot für konstanten Weight = 1 und verschiedene Bias-Werte
for bias in bias_values_2:
    ax2.plot([-2, -1, 0, 1, 2], [weight_constant * x + bias for x in [-2, -1, 0, 1, 2]], label=f"Bias = {bias}")
ax2.set_title("Verschiedene Bias bei konstantem Gewicht")
ax2.set_xlabel("x")
ax2.set_ylabel("y = weight * x + bias")
ax2.legend()
ax2.grid(True)
ax2.set_xticks(np.arange(-2,3,1))
ax2.set_yticks(np.arange(-4,7,1))
ax2.axhline(0, 2, color='black',linewidth=1)
ax2.axvline(0, 5, color='black',linewidth=1)
ax2.annotate('', xy=(2, 0), xytext=(0, 0), arrowprops=dict(facecolor='black', edgecolor='black', arrowstyle='->', lw=1))
ax2.annotate('', xy=(0, 5), xytext=(0, 0), arrowprops=dict(facecolor='black', edgecolor='black', arrowstyle='->', lw=1))

# Plot anzeigen
plt.tight_layout()
plt.show()

Sehen wir uns ein einfaches Beispiel in `Python` an, um einige der mathematischen Konzepte nachzuvollziehen, die bei neuronalen Netzwerken zum Einsatz kommen.

## Einfaches neuronales Netzwerk in `Python`

Wir versuchen zuerst die Eingabeschicht eines Netzwerks bestehend aus einem Neuron mit zwei Eingabewerten $x_1, x_2$, zwei Gewichten $w_1, w_2$ und einem Bias $b$ in `Python` zu modellieren. Dabei werden beim ersten Durchlauf des Netzwerks zuerst die Eingabewerte in die Eingabeschicht weitergegeben und die Gewichte und der Bias zufällig initialisiert.

### Vorwärtsdurchlauf des Netzwerks in `Python` (engl. Forwardpropagation)

Wir beginnen mit dem Erstellen der Eingabeschicht und modellieren die Berechnung des Outputs des ersten Neurons. Dies entspricht dem ersten Schritt der Forwardpropagation.

$$\text{Input} = \sum_{i=1}^2 x_i w_i + b = x_1 w_1 + x_2 w_2 + b$$

### Feedforward Propagation - Eingabeschicht mit einem Neuron

In [ ]:
# Eingabewerte
inputs = [5, 7]

# Zufällig initialisierte Gewichte
W1     = [6, 4]

# Zufällig initialisierter Bias
b1     = 3

# Aktivierung
A1 = inputs[0] * W1[0] + inputs[1] * W1[1] + b1
A1

Verallgemeinern wir unser Modell auf zwei Neuronen in der Eingabeschicht.

### Feedforward Propagation - Eingabeschicht mit zwei Neuronen

In [ ]:
### Abbildung - zwei Neuronen in der Eingabeschicht
# Slides
image_paths = ["./images/w_b1.png", "./images/w_b2.png", "./images/w_b3.png", "./images/w_b4.png", "./images/w_b5.png", "./images/w_b6.png", "./images/w_b7.png", "./images/w_b8.png"]

# Auswahl der Bilder
def show_image(index):
    img = mpimg.imread(image_paths[index])
    plt.figure(figsize=(2, 1), dpi = 600)
    plt.imshow(img)
    plt.axis('off')
    plt.show()

# Slider-Widget erstellen
slider = widgets.IntSlider(min=0, max=len(image_paths) - 1, step=1, description='Bild')
widgets.interactive(show_image, index=slider)


In [ ]:
# Eingabewerte
inputs = [5, 7]

# Zufällig initialisierte Gewichte für Inputs in Neuron 1
W1     = [6, 4]

# Zufällig initialisierter Bias für Inputs in Neuron 1
b1     = 3

# Zufällig initialisierte Gewichte für Inputs in Neuron 2
W2     = [5, 3]

# Zufällig initialisierter Bias für Inputs in Neuron 2
b2     = -5

A1 = inputs[0] * W1[0] + inputs[1] * W1[1] + b1
     
A2 = inputs[0] * W2[0] + inputs[1] * W2[1] + b2

print('Ausgabe von Neuron 1:', A1)
print('Ausgabe von Neuron 2:', A2)

Dies läßt sich vereinfachen, indem wir alle Gewichte einer Schicht zu einer Matrix zusammenfassen und die Inputs und Biases als Spaltenvektoren angeben. Wir berechnen also folgendes:

$$
A_1
=
W_1 \cdot \vec{X} + \vec{b_1}
=
\begin{pmatrix}
6 & 4 \\
5 & 3
\end{pmatrix}
\cdot
\begin{pmatrix}
5  \\
7 
\end{pmatrix}
+
\begin{pmatrix}
3  \\
-5 
\end{pmatrix}
=
\begin{pmatrix}
61 \\
41
\end{pmatrix}
$$

In [ ]:
# Eingabewerte
X = np.array([[5],[7]])
# Gewichte der Eingabeschicht als Matrix
W1 = np.array([[6, 4],[5, 3]])
# Biases der Eingabeschicht als Spaltenvektor
b1 = np.array([[3],[-5]])

In [ ]:
# Ausgabe versteckte Schicht
W1 @ X + b1

### Vektoren und Matrizen in `NumPy`

In diesem Abschnitt beschäftigen wir uns mit dem Framework `NumPy`. `NumPy` ist auf  numerische Berechnungen in `Python` spezialisiert und deckt viele Bereiche der Mathematik ab. Sehen wir uns an wie wir `NumPy` verwenden können, um lineare Algebra betreiben. 

### Vektoren

Das **Skalarprodukt** zweier Vektoren $\vec{a}=\begin{pmatrix}
1 \\
2 \\
3
\end{pmatrix}$ und $\vec{b}=\begin{pmatrix}
4 \\
5 \\
6
\end{pmatrix}$ ist gegeben durch:

$$\vec{a} \cdot \vec{b}=
\begin{pmatrix}
1 \\
2 \\
3
\end{pmatrix}
\cdot
\begin{pmatrix}
4 \\
5 \\
6
\end{pmatrix}
=
1 \cdot 4 + 2 \cdot 5 + 3 \cdot 6 = 4 + 10 + 18 = 32
$$

Wir können in `NumPy`-Arrays mit der Funktion `array()` erstellen, indem wir Zeilen und Spalten als Listen übergeben. Die beiden Vektoren $\vec{a}$ und $\vec{b}$ aus dem obigen Beispiel können wir folgendermaßen in `NumPy` anschreiben:

In [ ]:
a = np.array([1,2,3])

b = np.array([4,5,6])

In `NumPy` können wir das Skalarprodukt zweier Vektoren mit der Funktion `dot()` oder dem Symbol `@` berechnen:

In [ ]:
# Vektor- bzw. Matrizenmultiplikation
a @ b

*Hinweis*: Ein $n$-dimensionales Array wird in `NumPy` ohne weitere Spezifizierung nicht im mathematischen Sinn in Zeilen- oder Spaltenvektor unterschieden sondern nach den sogenannten Broadcasting Regeln interpretiert.

In [ ]:
vector = np.array([1,2,3,4])

In [ ]:
vector.shape

Um einen Vektor eindeutig in `NumPy` festzulegen können wir mit zusätzlichen eckigen Klammern die fehlende Dimension angeben.

#### Beispiel: Spaltenvektor

$
\vec{a}
= 
\begin{pmatrix}
1 \\
2 \\
3 \\
4
\end{pmatrix}
$

In [ ]:
vector_column = np.array([[1],[2],[3],[4]])

In [ ]:
vector_column.shape

#### Beispiel: Zeilenvektor

$
\vec{b}
= 
\begin{pmatrix}
1 & 2 & 3 & 4
\end{pmatrix}
=
\vec{a}^T
$

In [ ]:
vector_row = np.array([[1, 2, 3, 4]])

In [ ]:
vector_row.shape

Wir können Vektoren in `NumPy` mit der Syntax `vector.T` transponieren:

In [ ]:
vector_column.T.shape

### Aufgabe: 

Berechnen Sie das Skalarprodukt $\vec{a}^T \cdot \vec{b}$ für die Vektoren:

$
\vec{a}
=
\begin{pmatrix}
1 \\
2 \\
3
\end{pmatrix}$
,
$
\vec{b}
=
\begin{pmatrix}
4 \\
5 \\
6
\end{pmatrix}$

In [ ]:
a = np.array([[1,2,3]])

b = np.array([[4],[5],[6]])

In [ ]:
a @ b

### Matrizen

Eine Matrix setzt sich aus Spalten- beziehungsweise Zeilenvektoren zusammen, dabei besteht eine $(m \times n)$-Matrix aus $m$ Zeilen und $n$ Spalten. Das untere Beispiel stellt also eine $(3 \times 3)$ Matrix dar:

$$A =\begin{pmatrix}
1 & 2 & 3 \\
4 & 5 & 6 \\
7 & 8 & 9
\end{pmatrix}$$

Beachten Sie, dass Matrizen insofern eine Verallgemeinerung von Vektoren darstellen, da diese dem Spezialfall einer $(n \times 1)$ Matrix entsprechen. Wir können die oben angegebene Matrix $A$ in `NumPy` anschreiben, indem wir Zeilen und Spalten als Liste von Listen übergeben.

In [ ]:
a = np.array([[1],[2],[3]])

A = np.array([[1,2,3],
              [4,5,6],
              [7,8,9]])

### Multiplikation von Vektoren mit Matrizen

$\vec{a}=\begin{pmatrix}
1 \\
2 \\
3
\end{pmatrix}$ , $A =\begin{pmatrix}
1 & 2 & 3 \\
4 & 5 & 6 \\
7 & 8 & 9
\end{pmatrix}$

$A \cdot \vec{a} = \begin{pmatrix}
1 & 2 & 3 \\
4 & 5 & 6 \\
7 & 8 & 9
\end{pmatrix}
\cdot
\begin{pmatrix}
1 \\
2 \\
3
\end{pmatrix}=
\begin{pmatrix}
1 \cdot 1 + 2 \cdot 2 + 3 \cdot 3 \\
4 \cdot 1 + 5 \cdot 2 + 6 \cdot 3 \\
7 \cdot 1 + 8 \cdot 2 + 9 \cdot 3
\end{pmatrix}=
\begin{pmatrix}
14 \\
32 \\
50
\end{pmatrix}
$ 

In [ ]:
A @ a

### Dimension von Vektoren, Matrizen und Tensoren

### Matrizen

Matrizen besitzen Dimensionen entsprechend ihrer Anzahl an Zeilen $m$ und Spalten $n$:

#### Beispiel: $4 \times 4$ Matrix

$A =
\begin{pmatrix}
1 & 2 & 3 & 4 \\
5 & 6 & 7 & 8 \\
9 & 10 & 11 & 12 \\
13 & 14 & 15 & 16
\end{pmatrix}
$

In [ ]:
A_matrix = np.array([[1,2,3,4],
                    [5,6,7,8],
                    [9,10,11,12],
                    [13,14,15,16]])


print('A_matrix:')
print(A_matrix)
print('')
print('Dimensionen von A_matrix:',A_matrix.shape)

Ähnlich wie Vektoren können Matrizen auch durch das Vertauschen von Zeilen und Spalten transponiert werden:

$A^T=
\begin{pmatrix}
1 & 5 & 9 & 13 \\
2 & 6 & 10 & 14 \\
3 & 7 & 11 & 15 \\
4 & 8 & 12 & 16
\end{pmatrix}
$

In [ ]:
A_matrix.T

*Hinweis*: Damit Vektoren oder Matrizen miteinander multipliziert werden können muß der linke Vektor oder die linke Matrix gleich viele Spalten besitzen wie der rechte Vektor oder die rechte Matrix Zeilen. 

In [ ]:
# Multiplikation zwischen Vektor und Matrix ist nicht kommutativ
print('Dimensionen von a:', a.shape)
print('Dimensionen von A:', A.shape)
#a @ A

In [ ]:
# Allgemein gilt für zwei Matrizen A * B mit Dimensionen A = (m, n), B = (o, p),
# dass die Dimension der Spalten n von A gleich der Dimension der Zeilen o von B sein müssen
print('Dimensionen von a.T:', a.T.shape)
print('Dimensionen von A  :', A.shape)
np.dot(a.T,A)

#### Aufgabe: 

Erstellen Sie die zwei Matrizen $A = \begin{pmatrix}1 & 2 \\ 4 & 4 \end{pmatrix}$ und $B=\begin{pmatrix}5 & 6 \\ 7 & 8 \end{pmatrix}$ in `NumPy` und berechnen Sie $A \cdot B$ und $B \cdot A$. Gilt $A \cdot B = B \cdot A$?

check?

In [ ]:
A = np.array([[1, 2],[3, 4]])

B = np.array([[5, 6],[7, 8]])

In [ ]:
A @ B

In [ ]:
B @ A

## Einfaches neuronales Netzwerk mit Matrizen

Wir versuchen aus den bisherigen theoretischen Überlegungen ein neuronales Netz mit drei Eingabewerten, drei Neuronen in der der ersten versteckten Schicht, vier Neuronen in der zweiten versteckten Schicht und zwei Neuronen in der Ausgabeschicht zu erstellen.

### Feedforward Propagation - Neuronales Netzwerk mit mehreren Schichten in Matrixdarstellung

### Initialisieren der Gewichte und Biases

In der Praxis werden die Gewichte $W_i$ und Schwellenwerte $b_i$ eines neuronalen Netzes am Anfang der Trainingsphase mit Zufallswerten initialisiert. In `NumPy` ist es möglich mit der Funktion `random.rand(m, n)` ein Array der Dimension $(m \times n)$ gefüllt mit standardnormalverteilten Zufallszahlen zu erstellen. 

Wir erstellen also für dieses Netz einen Spaltenvektor mit den Eingabewerten $X$ der Dimension $(3 \times 1)$, die Matrix $W_1$ der Gewichte der ersten versteckten Schicht der Dimension $(3 \times 3$ und die zugehörigen Biases $b_1$ der Dimension $(3 \times 1)$, die Matrix $W_2$ der Gewichte der zweiten versteckten Schicht der Dimension $(4 \times 3$ und die zugehörigen Biases $b_2$ der Dimension $(4 \times 1)$ und die Matrix der Gewichte der Ausgabeschicht $W_3$ der Dimension $(2 \times 4)$ und die zugehörigen Biases $b_3$ der Dimension $(2 \times 1)$.

<img src="./images/3_3_4_2_netz.png" alt="drawing" width="80%"/>

In [ ]:
input_size = 3  # Eingabeschicht
W1_size    = 3  # 3 Neuronen in der 1.ten verborgenen Schicht
W2_size    = 4  # 4 Neuronen in der 2.ten verborgenen Schicht
W3_size    = 2  # 2 Ausgabeklassen

In [ ]:
X = np.array([[5],[7],[1]]) # Eingabewerte

In [ ]:
# Initialisiere zufällige Gewichte und Biases
W1 = np.random.rand(W1_size, input_size) - 0.5
b1 = np.random.randn(input_size, 1)
W2 = np.random.rand(W2_size, W1_size) - 0.5
b2 = np.random.rand(W2_size, 1) - 0.5
W3 = np.random.rand(W3_size, W2_size) - 0.5
b3 = np.random.rand(W3_size, 1) - 0.5

### Ausgabe versteckte Schicht $1$

In [ ]:
W1

In [ ]:
b1

In [ ]:
# Ausgabe der ersten versteckten Schicht
A1 = W1 @ X + b1
A1

In [ ]:
print('shape_inputs   :',X.shape, '\n')
print('shape_W1       :',W1.shape, '\n')
print('shape_b1       :',b1.shape, '\n')
print('shape_A1       :',A1.shape, '\n')

### Ausgabe versteckte Schicht $2$

In [ ]:
W2

In [ ]:
b2

In [ ]:
# Ausgabe der zweiten versteckte schicht

A2 = W2 @ A1 + b2
A2

In [ ]:
print('shape_A1       :',A1.shape, '\n')
print('shape_W2       :',W2.shape, '\n')
print('shape_b2       :',b2.shape, '\n')
print('shape_A2       :',A2.shape, '\n')

### Ausgabeschicht

In [ ]:
W3

In [ ]:
b3

In [ ]:
# Ausgabeschicht
A3 = W3 @ A2 + b3
A3

In [ ]:
print('shape_A2       :',A2.shape, '\n')
print('shape_W3       :',W3.shape, '\n')
print('shape_b3       :',b3.shape, '\n')
print('shape_A3       :',A3.shape, '\n')

### Aktivierungsfunktionen

Gehen wir kurz auf eine weitere Feinheit des neuronalen Netzwerks ein, die Aktivierungsfunktion. Diese modulieren das weitergegebene Signal und sorgen für nichtlineare Modulation des Signals.

Eine wesentliche Eigenschaft von Aktivierungsfunktionen ist, **nichtlineare Zusammenhänge** abbilden zu können. Würden wir nur die lineare gewichtete Summe der Eingaben verwenden, würden wir im Wesentlichen nur lineare Funktion miteinander verknüpfen, wodurch wieder lineare Funktionen entstehen. Betrachten wir zum Beispiel die zwei linearen Funktionen $f(g(x)) = 3 g(x) +1$ und $g(x) = 4 x +2$. Dann kann man die Verkettung dieser Funktionen $f(g(x))$ wie folgt schreiben:

$$ f(g(x)) = 3 g(x) + 1 = 3 (4 x + 2) + 1 = 12 x + 7 $$

Das Ergebnis ist wieder eine lineare Funktion! Hingegen ist ein ausreichend großes künstliches neuronales Netz mit nicht linearen Aktivierungsfunktionen in der Lage, jede stetige Funktion zu approximieren.

Eine häufig verwendete Aktivierungsfunktion ist in diesem Zusammenhang die aus der logistischen Regression bekannte **[Sigmoidfunktion](https://de.wikipedia.org/wiki/Sigmoidfunktion)**:

$$A(z) = \frac{1}{1 + e^{-z}} $$

Dabei ist $A(z)$ die Aktivierung des Neurons und $z = \sum_i w_i x_i +b$ die gewichtete Summe der Inputs. 

Die Sigmoidfunktion wird sowohl in versteckten Schichten als auch als Ausgabeaktivierungsfunktion in **binären Klassifikationsnetzwerken** eingesetzt.

Eine der Sigmoidfunktion ähnliche Aktivierungsfunktion ist der **[Tangens hyperbolicus](https://de.wikipedia.org/wiki/Tangens_hyperbolicus_und_Kotangens_hyperbolicus)**:

$$\tanh (z) = \frac{\sinh (z)}{\cosh (z)} = \frac{e^z - e^{-z}}{e^z + e^{-z}}$$

Beide Funktionen haben gemeinsam, den Wertebereich, auf den sie abbilden, einzuschränken. Im Fall der logistischen Funktion bildet diese beliebige Werte aus $\mathbb{R}$ auf das Intervall $[ 0  \ $ ,$ \ 1 ]$. Beim des Tangens hyperbolicus bildet dieser Werte aus $\mathbb{R}$ auf das Intervall $[ -1  \ $ ,$ \ 1 ]$ ab.

Die Berechnungen der Sigmoid- als auch der Tangens-hyperbolicus-Funktion sind vergleichsweise rechenzeitintensive Operationen, da in ihnen die Terme $e^{\pm z}$ berechnet und dividiert werden müssen. 

Als Alternative dient die **[Gleichrichterfunktion](https://de.wikipedia.org/wiki/Rectifier_(neuronale_Netzwerke))** (*engl. rectified linear unit, ReLU*). Diese hat den Vorteil, dass sie einfacher als einige andere Aktivierungsfunktionen wie die Sigmoidfunktion oder die Tangens-hyperbolicus-Funktion zu berechnen ist. Sie trägt auch zur Vermeidung des Problems des verschwindenden Gradienten bei, das bei tiefen neuronalen Netzwerken auftreten kann. Die **ReLU-Funktion** ist wie folgt definiert:

$$ A(z) = max(0,z) \begin{cases}
z & \text{für} \ z \gt 0, \\ 0 & \text{sonst}
\end{cases} $$

Obwohl für $z < 0$ bei den Werten der ReLU-Funktion keine Steigung existiert, zeigen ReLU-Aktivierungsfunktionen in künstlichen neuronalen Netzwerken eine sehr gute Optimierungsleistung und sind inzwischen eine der am häufigsten eingesetzten Aktivierungsfunktionen in tiefen neuralen Netzwerken.

Eine Erweiterung der ReLU-Funktion stellt die **[Leaky-ReLU-Funktion](https://en.wikipedia.org/wiki/Rectifier_(neural_networks)#Leaky_ReLU)** dar. Die Idee dahinter ist, dass auch negative Werte für $z$ eine geringe Steigung aufweisen, um das sogenannte **[Vanishing-Gradient-Problem](https://en.wikipedia.org/wiki/Vanishing_gradient_problem)** zu umgehen. Die Leaky-ReLU-Funktion ist wie folgt definiert:

$$ A(z) \begin{cases}
z & \text{für} \ z \gt 0, \\ 0,01 \cdot z & \text{sonst}
\end{cases} $$

Eine weitere wichtige Aktivierungsfunktion für die **Ausgabeschicht von Klassifikationsnetzwerken** ist die **[Softmax-Funktion](https://de.wikipedia.org/wiki/Softmax-Funktion)**. Diese dient dazu, bei **Multiklassen-Klassifikation** die Wahrscheinlichkeitsverteilung der $K$ in unterschiedlichen möglichen Klassen zu berechnen. Die Softmax-Funktion ist in *Komponentenschreibweise* wie folgt definert:

$$ A(z_j) = \frac{e^{z_j}}{\sum_{k=1}^K e^{z_k}} $$

Für drei Klassen wäre die **Softmax-Funktion** zum Beispiel:

$$ A(z_1) = \frac{e^{z_1}}{e^{z_1}+e^{z_2}+e^{z_3}} \ , \   A(z_2) = \frac{e^{z_2}}{e^{z_1}+e^{z_2}+e^{z_3}} \ , \   A(z_3) = \frac{e^{z_3}}{e^{z_1}+e^{z_2}+e^{z_3}}$$

Die Summe der einzelnen Komponenten addiert sich dabei im Sinne einer Wahrscheinlichkeit der Zugehörigkeit zu einer von $j$-Klassen zu $1$ auf:

$$ A(z_1) +  A(z_2)+  A(z_3) = \frac{e^{z_1}}{e^{z_1}+e^{z_2}+e^{z_3}} + \frac{e^{z_2}}{e^{z_1}+e^{z_2}+e^{z_3}}+ \frac{e^{z_3}}{e^{z_1}+e^{z_2}+e^{z_3}} =  1 $$

Im Unterschied zur Ausgabeaktivierung von Netzwerken zur Klassifikation wird in der **Ausgabeschicht von Regressionsnetzwerken** eine **lineare Aktivierungsfunktion** verwendet, um kontinuierliche Werte zu erhalten, die auf kein bestimmtes Intervall beschränkt sind. Die lineare Aktivierungsfunktion kann wie folgt geschrieben werden:

$$A(z) = z$$

In der folgenden Abbildung sind die wichtigsten **Aktivierungsfunktionen** nochmals zusammengefasst dargestellt.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Werte generieren
x = np.linspace(-6, 6, 100)
sigmoid = 1 / (1 + np.exp(-x))
relu = np.maximum(0, x)
tanh = np.tanh(x)
linear = x
softmax = np.exp(x) / np.sum(np.exp(x), axis=0)
leaky_relu = np.where(x > 0, x, 0.01 * x)  # Leaky ReLU mit Alpha = 0.01

# Plot erstellen
plt.figure(dpi=600, figsize=(6, 3))
plt.rcParams.update({'font.size': 6})
plt.tight_layout()

# Sigmoid-Aktivierungsfunktion
plt.subplot(2, 3, 1)
plt.plot(x, sigmoid, label="Sigmoid", color="b")
plt.xlabel("Eingabe")
plt.ylabel("Ausgabe")
plt.title("Sigmoid-Aktivierungsfunktion")
plt.grid(True)
plt.legend()

# ReLU-Aktivierungsfunktion
plt.subplot(2, 3, 2)
plt.plot(x, relu, label="ReLU", color="r")
plt.xlabel("Eingabe")
plt.ylabel("Ausgabe")
plt.title("ReLU-Aktivierungsfunktion")
plt.grid(True)
plt.legend()

# Tanh-Aktivierungsfunktion
plt.subplot(2, 3, 3)
plt.plot(x, tanh, label="Tanh", color="g")
plt.xlabel("Eingabe")
plt.ylabel("Ausgabe")
plt.title("Tanh-Aktivierungsfunktion")
plt.grid(True)
plt.legend()

# Lineare Aktivierungsfunktion
plt.subplot(2, 3, 4)
plt.plot(x, linear, label="Lineare", color="m")
plt.xlabel("Eingabe")
plt.ylabel("Ausgabe")
plt.title("Lineare Aktivierungsfunktion")
plt.grid(True)
plt.legend()

# Softmax-Aktivierungsfunktion
plt.subplot(2, 3, 5)
plt.plot(x, softmax, label="Softmax", color="c")
plt.xlabel("Eingabe")
plt.ylabel("Ausgabe")
plt.title("Softmax-Aktivierungsfunktion")
plt.grid(True)
plt.legend()

# Leaky ReLU-Aktivierungsfunktion
plt.subplot(2, 3, 6)
plt.plot(x, leaky_relu, label="Leaky ReLU", color="y")
plt.xlabel("Eingabe")
plt.ylabel("Ausgabe")
plt.title("Leaky ReLU-Aktivierungsfunktion (Alpha=0.01)")
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

## Einfaches neuronales Netzwerk mit Aktivierungsfunktion

Wenden wir Aktivierungsfunktionen auf das oben skizzierte Netzwerk an. Dabei verwenden wir in dersten und zweiten versteckten Schicht **ReLU-Aktivierung** und in der Ausgabeschicht **Sigmoid-Aktivierung**.

In [ ]:
def relu_func(x):
    return np.maximum(0, x)

In [ ]:
def sigmoid(X):
    return (1/(1 + np.exp(1)**(-X)))

### Feedforward Propagation

### Initialisieren der Gewichte und Biases

Wir versuchen aus den bisherigen theoretischen Überlegungen ein neuronales Netzwerk zu erstellen.

In [ ]:
input_size = 3  # Eingabeschicht
W1_size    = 3  # 3 Neuronen in der 1.ten verborgenen Schicht - ReLU-Aktivierung
W2_size    = 4  # 4 Neuronen in der 2.ten verborgenen Schicht - ReLU-Aktivierung
W3_size    = 2  # 2 Ausgabeklassen - Sigmoidaktivierung

In [ ]:
X= np.array([[5],[7],[1]]) # Eingabewerte

In [ ]:
# Initialisiere zufällige Gewichte und Biases
W1 = np.random.rand(W1_size, input_size) - 0.5
b1 = np.random.randn(input_size, 1)
W2 = np.random.rand(W2_size, W1_size) - 0.5
b2 = np.random.rand(W2_size, 1) - 0.5
W3 = np.random.rand(W3_size, W2_size) - 0.5
b3 = np.random.rand(W3_size, 1) - 0.5

### Ausgabe versteckte Schicht $1$

In [ ]:
W1

In [ ]:
b1

In [ ]:
# Ausgabe der ersten versteckten Schicht mit ReLU-Aktivierung
A1 = relu_func(W1 @ X + b1)
A1

### Ausgabe versteckte Schicht $2$

In [ ]:
W2

In [ ]:
b2

In [ ]:
# Ausgabe der zweiten versteckte schicht mit ReLU-Aktivierung
A2 = relu_func(W2 @ A1 + b2)
A2

### Ausgabeschicht

In [ ]:
W3

In [ ]:
b3

In [ ]:
# Ausgabeschicht mit Sigmoidaktivierung
A3 = sigmoid(W3 @ A2 + b3)
A3

## Numerische Ableitung

In Vorbereitung auf den **Backpropagation-Algorithmus** beschäftigen wir uns kurz mit numerischer Ableitung von Funktionen.

Sehen wir uns als Beispiel die Funktion $f(x) = x^5 + x^3 + x$ und ihre Ableitung an:

In [ ]:
def f(x):
    return x**5 + x**3 + x

In [ ]:
### Abbildung f(x) = x**5 + x**3 + x
plt.figure(dpi=600, figsize=(6, 3))
plt.rcParams.update({'font.size': 6})
plt.grid()
plt.tight_layout()
x = np.linspace(-2,2, 1000)
y = f(x)


_ = plt.plot(x,y)

### Differenzenquotient

Um die Funktion numerisch abzuleiten können wir als einfachste Annäherung den **Differenzenquotienten** $\frac{ f (x + \epsilon) - f (x)}{(x + \epsilon) - x }$ der Funktion an den Stellen $x$ und $x + \epsilon$ verwenden, wobei $\epsilon$ die Schrittweite bezeichnet.

$$f^{\prime}(x) = \frac{d f (x)}{d x}  \approx \frac{ f (x + \epsilon) - f (x)}{(x + \epsilon) - x } = \frac{ f (x + \epsilon) - f (x)}{\epsilon} $$

$$eg.: f^{\prime}(x) = (x^2)^{\prime} = 2x  \approx \frac{ (x + \epsilon)^2 - (x)^2}{\epsilon} $$

Wir können dies in `Python` folgendermaßen anschreiben:

In [ ]:
def derivative(f, x, delta):
    return (f(x + delta) - f(x))/delta

In [ ]:
### Abbildung f(x), f'(x)
plt.figure(dpi=600, figsize=(6, 3))
plt.grid()
plt.xlim([-2.2,2.2])
plt.ylim([-2.2,7.2])
plt.yticks(np.arange(-2, 8, 1))
plt.xticks(np.arange(-2, 3, 1))
plt.rcParams.update({'font.size': 6})
plt.tight_layout()
plt.plot(x,y)
_ = plt.plot(x, derivative(f, x, 10**-5))

### Ableitung der ReLU-Aktivierungsfunktion

Die ReLUfunktion ist gegeben durch $f_{ReLU}(x) = max(0, x)$. Versuchen wir diese wichtige und vielleicht etwas unintuitive Funktion mit unserer Funktion `derivative()` numerisch abzuleiten.

In [ ]:
def relu_func(x):
    return np.maximum(0, x)

In [ ]:
derivative(relu_func, x, delta = 1e-5)

In [ ]:
# Plot
plt.figure(dpi=600, figsize=(6, 3))
plt.rcParams.update({'font.size': 6})
plt.tight_layout()
plt.plot(x, derivative(relu_func, x, delta = 1e-5), linewidth = 2.5)
plt.xlabel("x")
plt.ylabel("H(x)")
plt.title("Heaviside-Sprungfunktion als Ableitung der ReLU-Funktion")
plt.ylim([-0.5,1.5])
plt.grid(True)
plt.show()

### Problem der numerischen Ableitung - unstetige Funktionen

In [ ]:
### Abbildung ReLU-Funktion und Ableitung der ReLU-Funktion
# ReLU-Funktion und numerische Ableitung
def relu_func(x):
    return np.maximum(0, x)

def derivative(func, x, delta=1e-5):
    return (func(x + delta) - func(x)) / (delta)

# Funktion, die den Plot aktualisiert
def plot_reAct(num_points):
    x = np.linspace(-2, 2, num_points)  # Anzahl der Punkte anpassen

    plt.figure(dpi=600, figsize=(6, 3))
    plt.rcParams.update({'font.size': 6})
    plt.tight_layout()

    # ReLU-Funktion
    plt.subplot(2, 1, 1)
    plt.plot(x, relu_func(x), label='ReLU(x)', color='blue')
    plt.title('ReLU-Funktion')
    plt.xlabel('x')
    plt.ylabel('ReLU(x)')
    plt.xlim(-2, 2)
    plt.grid(True)
    plt.legend()

    # Numerische Ableitung der ReLU-Funktion
    plt.subplot(2, 1, 2)
    plt.plot(x, derivative(relu_func, x, delta=1e-5), label="ReLU'(x)", color='red')
    plt.title('Numerische Ableitung der ReLU-Funktion')
    plt.xlabel('x')
    plt.ylabel("ReLU'(x)")
    plt.xlim(-2, 2)
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

# Schieberegler für die Anzahl der dargestellten Punkte
interactive_plot = interactive(plot_reAct, num_points=(70, 1000, 1))  # Schieberegler von 70 bis 1000 Punkten
interactive_plot

Eine einfachere Lösung besteht darin nicht die tatsächliche Ableitung zu bilden, sondern die Ableitung über die Heaviside-Funktion festzulegen.

In [ ]:
def derivative_ReLU(Z):
    return np.where(Z > 0, 1, 0) 

In [ ]:
x = np.linspace(-2,2, 1000)
plt.title('Die Heaviside-Funktion als Ableitung der ReLU-Funktion')
plt.xlabel('x')
plt.ylabel("ReLU'(x)")
_ = plt.plot(x, derivative_ReLU(x), color = 'red')

## Gradientenabstiegsverfahren (engl. Gradient Descent)

Beim klassischen **[Gradientenverfahren](https://de.wikipedia.org/wiki/Gradientenverfahren)** geht es darum, im Idealfall ein globales Maximum/Minimum einer gegebenen Funktion zu bestimmen, man spricht auch von einem sogenannten **[Optimierungsproblem](https://de.wikipedia.org/wiki/Optimierung_(Mathematik))**. Die Vorgehensweise ist dabei die Bildung des namensgebenden **[Gradienten](https://de.wikipedia.org/wiki/Gradient)** einer Funktion für alle unabhängigen Parameter. Dabei zeigt der Gradient in die Richtung des größten Anstiegs der Funktion. Um zum Minimum der Funktion zu kommen, wählen wir einen Startwert $w_{\text{alt}}$, der als Ausgangspunkt des iterativen Abstiegs zum Minimum dient, und gehen bei jedem Schritt des Gradientenverfahrens in Richtung des negativen Gradienten der Funktion, indem wir den Gradienten mal einer Lernrate $\alpha$ von $w_{\text{alt}}$ abziehen, um $w_{\text{neu}}$ zu berechnen. Allgemein kann man schreiben:

$$w_{\text{neu}} = w_{\text{alt}} - \alpha \cdot \nabla f(w) $$

Dabei ist $\nabla$ der **[Nabla-Operator](https://de.wikipedia.org/wiki/Nabla-Operator)**. Im eindimensionalen Fall entspricht er einfach der Ableitung nach der unabhängigen Variable.

Betrachen wir zum Beispiel, das Minimum der Funktion $f(x)= x^2$ zu bestimmen, um das Gradientenabstiegsverfahren zu verstehen.

In [ ]:
N = 50
x = np.linspace(-5, 5, N)
y = x**2
plt.figure(dpi=600, figsize=(6, 3))
plt.rcParams.update({'font.size': 6})
plt.tight_layout()
_ = plt.plot(x, y)

Wir bestimmen den Gradienten der Funktion $f(x)=x^2$:

$$\frac{df}{dx} = 2 x$$

Um das Minimum zu bestimmen, subtrahieren wir den Gradienten multipliziert mit einer Lernrate $\alpha$ von einem zufällig gewählten Startwert und iterieren so lange, bis der Gradient gegen $0$ konvergiert. Wir gehen somit in die Gegenrichtung des größten Zuwachses der Funktion mit jeder Iteration auf ein Minimum zu. In jedem Schritt berechnen wir den nächsten $x$-Wert mit:

$$x_{\text{neu}} = x_{\text{alt}} - \alpha \cdot \frac{df}{dx} = x_{\text{alt}} - \alpha \cdot 2 x$$

Sehen wir uns dazu ein Code-Beispiel an:

In [ ]:
x_alt = 5
alpha = 0.1

In [ ]:
for i in range(0, 30):
    x_neu = x_alt - alpha * (2 * x_alt)
    x_alt = x_neu
x_neu

Wie wir sehen können, konvergiert der Wert für $x_{\text{neu}}$ gegen das Minimum von $f(x)$ bei $x = 0$.

In der folgenden Abbildung sehen Sie die ersten drei Schritte beim Gradientenabstiegsverfahren mit Lernrate `alpha = 0.1` und ausgehend vom Startpunkt `x_alt = 5`. Beachten Sie, dass die Schrittweite abnimmt, je näher wir dem Minimum kommen.

In [ ]:
# Zielfunktion und deren Ableitung
def f(x):
    return x**2


def df(x):
    return 2 * x


# Startpunkt und Lernrate für den Gradientenabstieg
x_start = 5.0
alpha = 0.1

# Anzahl der Schritte
num_steps = 3
x_history = [x_start]

# Gradientenabstieg durchführen und die Pfeile zeichnen
for _ in range(num_steps):
    x_current = x_start
    x_start = x_start - alpha * df(x_start)
    x_history.append(x_start)

# X-Werte für die Funktion
x = np.linspace(-5, 5, 100)

# Diagramm erstellen
plt.figure(dpi=600, figsize=(6, 3))
plt.plot(x, f(x), label="f(x) = $x^2$", color="blue")
_ = plt.scatter(
    x_history, [f(x) for x in x_history], color="red", label="Gradient Descent Steps"
)

# Pfeile zeichnen, die die Schritte des Gradientenabstiegs verbinden
for i in range(1, len(x_history)):
    dx = x_history[i] - x_history[i - 1]
    dy = f(x_history[i]) - f(x_history[i - 1])
    plt.quiver(
        x_history[i - 1],
        f(x_history[i - 1]),
        dx,
        dy,
        angles="xy",
        scale_units="xy",
        scale=1,
        color="green",
        width=0.0075,
        headaxislength=4,
        headlength=4,
    )

plt.xlabel("x")
plt.ylabel("f(x)")
plt.legend()
plt.title("Gradient Descent")
plt.grid(True)
plt.rcParams.update({'font.size': 6})
plt.tight_layout()
plt.show()

Die Konvergenzgeschwindigkeit (die notwendige Anzahl an Schritten) ist dabei von der Lernrate und dem zufällig gewählten Startpunkts abhängig. Dabei gilt es, einen Mittelweg zwischen einer **zu kleinen Lernrate**, die zu einer unnötig **hohen Anzahl von Schritten** führt, und einer **zu hohen Lernrate**, die zu **oszillierenden Lösungen** führt, da sie immer wieder über das Minimum hinweg springt, zu finden. Oft ist das Finden der besten Lernrate ein iterativer Prozess des Experimentierens. Sie können mit verschiedenen Lernrateneinstellungen beginnen (z.B. $0.1$, $0.01$, $0.001$) und die Leistung auf einem Validierungsdatensatz überwachen.

Versuchen wir jetzt das Gradientenabstiegsverfahren anzuwenden um ein neuronales Netz zu trainieren und betrachten dazu das folgende Beispiel.

# Backpropagation

Sehen wir uns ein Beispiel für den **Backpropagation-Algorithmus** Schritt für Schritt an. Aus Gründen der Übersichtlichkeit verwenden wir dabei ein Netz mit zwei Eingaben, einer versteckten Schicht mit zwei Neuronen und **linearer Aktivierung** und einer Ausgabeschicht mit **Sigmoid-Aktivierung**.

## Forwardpropagation

Zuerst führen wir für das in der Abbildung gezeigte neuronale Netz wie gewohnt eine Forwardpropagation durch.

<img src="./images/backprop_forward.png" alt="drawing" width="80%"/>

### Anfangswerte

Wir gehen von folgenden Ausgangsparametern für Gewichte und Biases aus:

In [ ]:
X1 = 0.85

X2 = 0.5

w1 = 0.75

w2 = 0.55

w3 = 0.05

w4 = 0.05

w5 = 0.05

w6 = 0.015

w7 = 0.85

w8 = 0.95

b1 = 0.25

b2 = 0.15

b3 = 0.15

b4 = 0.25

Wie verwenden für die Eingabe- und die versteckte Schicht lineare Aktivierung und für die Ausgabeschicht die Sigmoidaktivierungsfunktion.

In [ ]:
def sigmoid(X):
    return (1/(1 + np.exp(1)**(-X)))

In [ ]:
def linear_activation(x):
    return x

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6), dpi = 600)
x_werte = np.linspace(-20,20,1000)
ax.set_title('Sigmoidfunktion', fontsize = 14)
_ = ax.plot(x_werte, sigmoid(x_werte))

Die Ableitung der Sigmoidfunktion ist gegeben durch:

$S(x)^{\prime} = S(x) (1 - S(x)) $

Wir können dies überprüfen, indem wir mit unserer Funktion `derivative()` die Sigmoidfunktion ableiten und das Ergebnis gleichzeitig mit dem oberen Ausdruck plotten.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6), dpi = 600)
ax.set_title('Ableitung der Sigmoidfunktion', fontsize = 14)
#plt.plot(x_werte, sigmoid(x_werte)*(1 - sigmoid(x_werte)))
_ = ax.plot(x_werte, derivative(sigmoid, x_werte, 10**-12))
plt.show()

Wir berechnen den Vorwärtsdurchlauf durch das Netz bis zu den Aktivierungen $A_3$, $A_4$ in der Ausgabeschicht.

In [ ]:
Z1 = w1 * X1 + w2 * X2 + b1
Z1

In [ ]:
Z2 = w3 * X1 + w4 * X2 + b2
Z2

In [ ]:
A1 = linear_activation(Z1)
A1

In [ ]:
A2 = linear_activation(Z2)
A2

#### Aktivierung im Ausgabeneuron $1$: $A_3$

In [ ]:
Z3 = w5 * A1 + w6 * A2 + b3
Z3

In [ ]:
A3 = sigmoid(Z3)
A3

#### Aktivierung im Ausgabeneuron $2$: $A_4$

In [ ]:
Z4 = w7 * A1 + w8 * A2 + b4
Z4

In [ ]:
A4 = sigmoid(Z4)
A4

### Verlustfunction - Mean Squared Error (MSE) 

Um den Fehler der Vorhersage zu dem tatsächlichen Wert der Ausgabe zu bestimmen verwenden wir den 

Zuerst müssen wir ein Maß für den Fehler des Modells einführen. Ein Möglichkeit den Fehler eines Modells zu bewerten ist durch den **[MSE (Mean Squared Error)](https://de.wikipedia.org/wiki/Mittlere_quadratische_Abweichung)** gegeben. Allgemein spricht man von **[Verlustfunktionen](https://de.wikipedia.org/wiki/Verlustfunktion_(Statistik))** (*engl. loss function*).

$$ MSE = E_{total} = \frac{1}{n} \sum_{i=1}^n (y_i - \hat y_i)^2 $$

Dabei ist $y_i$ die erwartete Ausgabe (*engl. ground truth*) und $\hat y_i$ die Vorhersage des Modells (*engl. prediction*). Wir nehmen an $\hat y_1 = 0.01$ und $\hat y_1 = 0.99$ gegeben sind.

Der Index $n$ läuft dabei über alle Neuronen der Ausgabeschicht. In unserem Beispiel gilt also $E_{total} = E_1 + E_2$:

$E_1 = \frac{1}{2} (\hat y_1 - A_3 )^2 = \frac{1}{2} (0.01 - A_3 )^2$

In [ ]:
# Fehler Ausgabeschicht - erstes Neuron
E_1 = 1/2*(0.01 - A3)**2 
E_1

$E_2 = \frac{1}{2} (\hat y_2 - A_4 )^2 = \frac{1}{2} (0.99 - A_4)^2$

In [ ]:
# Fehler Ausgabeschicht - zweites Neuron
E_2 = 1/2*(0.99 - A4)**2 
E_2

$E_{total} = E_1 + E_2$

In [ ]:
E_total = E_1 + E_2
E_total

## Backpropagation

Wir haben also den Gesamtfehler der Ausgabe durch $E_{total}$ gegeben.

Wenden wir uns jetzt dem Optimierungsverfahren dieses Fehlers, dem **[Backpropagation-Algorithmus](https://de.wikipedia.org/wiki/Backpropagation)**, zu. Dafür greifen wir auf die von uns zuvor besprochenen Grundlagen wie das **Gradientenverfahren** und **Aktivierungsfunktionen** zurück und versuchen systematisch die Gewichte des Netzwerks so anzupassen um bessere Vorhersagen zu erhalten.

Dabei gehen wir von den Ausgaben des Neuronalen Netzes rückwärts und passen jeweils die Gewichte und Biases in Richtung des Gradientenabstiegsverfahrens an. Um die Abhängigkeiten des Fehlers (der Verlustfunktion) zu berechnen müssen wir dabei die Kettenregel anwenden.

Zur Erinnerung sehen wir uns die Anwendung der Kettenregel an einem Beispiel an:

### Kettenregel

$f(g(x))^{\prime} = \frac{df}{dg}\frac{dg}{dx}$

e.g.:

$f = g^2, g = sin(x)$

$\frac{d}{dx}(sin(x))^2 = 2 \cdot sin(x) \cdot cos(x)$

## Ausgabeschicht

Berechenen wir ausgehend von der Ausgabeschicht di Anpassung der Gewichte ($w_5, w_6, w_7, w_8$) der Ausgabeschicht: 

<img src="./images/backprop_w5_f2.png" alt="drawing" width="80%"/>

### Änderung von $E_{total}$ nach $w_5$ 

#### Gesamte Ableitung: $\frac{\partial E_{total}}{\partial w_5} = \frac{\partial E_{total}}{\partial A_3} \frac{\partial A_3}{\partial Z_3} \frac{\partial Z_3}{\partial w_5}   $

#### 1.)

#### $\frac{\partial E_{total}}{\partial A_3} = \frac{\partial }{\partial A_3}(E_1 + E_2 ) = \frac{\partial }{\partial A_3}(\frac{1}{2} (\hat y_1 - A_3 )^2 + \frac{1}{2} (\hat y_2 - A_4 )^2 ) = \frac{\partial }{\partial A_3}(\frac{1}{2} ( 0.01 - A_3 )^2 + \frac{1}{2} (0.99 - A_4 )^2 )  \\ = -(0.01 - A_3)$

In [ ]:
dE_total_nach_A3 = -(0.01 - A3)
dE_total_nach_A3

#### 2.)

####  $\frac{\partial A_3}{\partial Z_3} = -\frac{\partial }{\partial Z_3} \frac{1}{1 + e^{-Z_3}} = A_3 (1 - A_3)$

In [ ]:
dA3_nach_Z3 = A3 * (1 - A3)
dA3_nach_Z3

#### 3.)

#### $\frac{\partial Z_3}{\partial w_5} = \frac{\partial }{\partial w_5}(w_5 \cdot A_1 + w_6 \cdot A_2 + b_3) = A_1$

In [ ]:
dZ3_nach_w5 = A1
dZ3_nach_w5

#### Gesamte Ableitung: $\frac{\partial E_{total}}{\partial w_5} = \frac{\partial E_{total}}{\partial A_3} \frac{\partial A_3}{\partial Z_3} \frac{\partial Z_3}{\partial w_5}   $

In [ ]:
dE_total_nach_w5 = dE_total_nach_A3 * dA3_nach_Z3 * dZ3_nach_w5
dE_total_nach_w5

#### Gewicht $w_5$ anpassen

$w_{5 neu} = w_5 - \alpha \cdot \frac{\partial E_{total}}{\partial w_5}$, $\alpha = 0.5 \cdots \text{Learning rate}$

In [ ]:
w5_neu = w5 - 0.5 * dE_total_nach_w5
w5_neu

In [ ]:
# Änderung für w5
delta_w5 = w5_neu - w5
delta_w5

### Änderung von $E_{total}$ nach $w_6$ 

<img src="./images/backprop_w6_f3.png" alt="drawing" width="80%"/>

#### Gesamte Ableitung: $\frac{\partial E_{total}}{\partial w_6} = \frac{\partial E_{total}}{\partial A_3} \frac{\partial A_3}{\partial Z_3} \frac{\partial Z_3}{\partial w_6}   $

#### 1.)

#### $\frac{\partial E_{total}}{\partial A_3} = -(0.01 - A_3)$

In [ ]:
dE_total_nach_A3 = -(0.01 - A3)
dE_total_nach_A3

#### 2.)

####  $\frac{\partial A_3}{\partial Z_3} = -\frac{\partial }{\partial Z_3} \frac{1}{1 + e^{-Z_3}} = A_3 (1 - A_3)$

In [ ]:
dA3_nach_Z3 = A3 * (1 - A3)
dA3_nach_Z3

#### 3.)

#### $\frac{\partial Z_3}{\partial w_6} = \frac{\partial }{\partial w_6}(w_5 \cdot A_1 + w_6 \cdot A_2 + b_3) = A_2$

In [ ]:
dZ3_nach_w6 = A2
dZ3_nach_w6

#### Gesamte Ableitung: $\frac{\partial E_{total}}{\partial w_6} = \frac{\partial E_{total}}{\partial A_3} \frac{\partial A_3}{\partial Z_3} \frac{\partial Z_3}{\partial w_6}   $

In [ ]:
dE_total_nach_w6 = dE_total_nach_A3 * dA3_nach_Z3 * dZ3_nach_w6
dE_total_nach_w6

#### Gewicht $w_6$ anpassen

In [ ]:
w6_neu = w6 - 0.5 * dE_total_nach_w6
w6_neu

In [ ]:
# Änderung für w6
delta_w6 = w6_neu - w6
delta_w6

### Änderung von $E_{total}$ nach $w_7$ 

<img src="./images/backprop_w7_f.png" alt="drawing" width="80%"/>

#### Gesamte Ableitung: $\frac{\partial E_{total}}{\partial w_7} = \frac{\partial E_{total}}{\partial A_4} \frac{\partial A_4}{\partial Z_4} \frac{\partial Z_4}{\partial w_7}   $

#### 1.)

#### $\frac{\partial E_{total}}{\partial A_4}  =\frac{\partial }{\partial A_4}(\frac{1}{2} (\hat y_1 - A_3 )^2 + \frac{1}{2} (\hat y_2 - A_4 )^2 ) \\ = \frac{\partial }{\partial A_4}(\frac{1}{2} ( 0.01 - A_3 )^2 + \frac{1}{2} (0.99 - A_4 )^2 )  \\ = -(0.99 - A_4)$

In [ ]:
dE_total_nach_A4 = -(0.99 - A4)
dE_total_nach_A4

#### 2.)

####  $\frac{\partial A_4}{\partial Z_4} = \frac{\partial }{\partial Z_4} \frac{1}{1 + e^{-Z_4}} = A_4 (1 - A_4)$

In [ ]:
dA4_nach_Z4 = A4 * (1 - A4)
dA4_nach_Z4

#### 3.)

#### $\frac{\partial Z_4}{\partial w_7} = \frac{\partial }{\partial w_7}(w_7 \cdot A_1 + w_8 \cdot A_2 + b_4) = A_1$

In [ ]:
dZ4_nach_w7 = A1
dZ4_nach_w7

In [ ]:
dE_total_nach_w7 = dE_total_nach_A4 * dA4_nach_Z4 * dZ4_nach_w7
dE_total_nach_w7

#### Gewicht $w_7$ anpassen

In [ ]:
w7_neu = w7 - 0.5 * dE_total_nach_w7
w7_neu

In [ ]:
# Änderung für w7
delta_w7 = w7_neu - w7
delta_w7

### Änderung von $E_{total}$ nach $w_8$ 

<img src="./images/backprop_w8_f.png" alt="drawing" width="80%"/>

#### Gesamte Ableitung: $\frac{\partial E_{total}}{\partial w_8} = \frac{\partial E_{total}}{\partial A_4} \frac{\partial A_4}{\partial Z_4} \frac{\partial Z_4}{\partial w_8}   $

#### 1.)

#### $\frac{\partial E_{total}}{\partial A_4} = -(0.99 - A_4)$

In [ ]:
dE_total_nach_A4 = -(0.99 - A4)
dE_total_nach_A4

#### 2.)

####  $\frac{\partial A_4}{\partial Z_4} = \frac{\partial }{\partial Z_4} \frac{1}{1 + e^{-Z_4}} = A_4 (1 - A_4)$

In [ ]:
dA4_nach_Z4 = A4 * (1 - A4)
dA4_nach_Z4

#### 3.)

#### $\frac{\partial Z_4}{\partial w_8} = \frac{\partial }{\partial w_8}(w_7 \cdot A_1 + w_8 \cdot A_2 + b_2) = A_2$

In [ ]:
dZ4_nach_w8 = A2
dZ4_nach_w8

In [ ]:
dE_total_nach_w8 = dE_total_nach_A4 * dA4_nach_Z4 * dZ4_nach_w8
dE_total_nach_w8

#### Gewicht $w_8$ anpassen

In [ ]:
w8_neu = w8 - 0.5 * dE_total_nach_w8
w8_neu

In [ ]:
# Änderung für w8
delta_w8 = w8_neu - w8
delta_w8

## Versteckte Schicht

In der versteckten Schicht ergibt sich eine zusätzliche Abhängigkeit da sowohl $E_1$ als auch $E_2$ von $A_1$ und $A_2$ abhängen, ergibt sich $E_{total}$ zu: 

### Änderung von $E_{total}$ nach $w_1$ 

<img src="./images/backprop_w1_f.png" alt="drawing" width="80%"/>

$\frac{\partial E_{total}}{\partial w_1} = \frac{\partial E_{total}}{\partial A_1} \frac{\partial A_1}{\partial Z_1} \frac{\partial Z_1}{\partial w_1}   $

$\frac{\partial E_{total}}{\partial A_1} = \frac{\partial E_1}{\partial A_1} +  \frac{\partial E_2}{\partial A_1} $

weil gilt:

$E_1 = \frac{1}{2} (\hat y_1 - A_3 )^2 \\ = \frac{1}{2} (0.01 - A_3 )^2 \\ = \frac{1}{2} (0.01 - sigmoid(Z3) )^2 \\ = \frac{1}{2} (0.01 - sigmoid(w_5 \cdot A_1 + w_6 \cdot A_2 + b_2) )^2 \\ = E_1 (A_1, A_2)$

und

$E_2 = \frac{1}{2} (\hat y_2 - A_4 )^2 \\ = \frac{1}{2} (0.99 - A_4 )^2 \\ = \frac{1}{2} (0.99 - sigmoid(Z4) )^2 \\ = \frac{1}{2} (0.99 - sigmoid(w_7 \cdot A_1 + w_8 \cdot A_2 + b_2) )^2 \\ = E_2 (A_1, A_2)$

#### 1.)

$\frac{\partial E_1}{\partial A_1} = \frac{\partial E_1}{\partial Z_3} \frac{\partial Z_3}{\partial A_1}$

Wir haben $\frac{\partial E_1}{\partial Z_3}$ bereits ausgerechnet da gilt:

$\frac{\partial E_1}{\partial Z_3} = \frac{\partial E_1}{\partial A_3} \frac{\partial A_3}{\partial Z_3}$

In [ ]:
dE1_nach_Z3 = dE_total_nach_A3 * dA3_nach_Z3
dE1_nach_Z3

$\frac{\partial Z_3}{\partial A_1}$ ergibt sich zu:

$\frac{\partial Z_3}{\partial A_1} = \frac{\partial }{\partial A_1} (w_5 A_1 + w_6 A_2 + b_2) = w_5$

In [ ]:
dZ3_nach_A1 = w5
dZ3_nach_A1

Insgesamt ergibt sich also für $\frac{\partial E_1}{\partial A_1} = \frac{\partial E_1}{\partial Z_3} \frac{\partial Z_3}{\partial A_1}$:

In [ ]:
dE1_nach_A1 = dE1_nach_Z3 * dZ3_nach_A1
dE1_nach_A1

#### 2.)

$\frac{\partial E_2}{\partial A_1} = \frac{\partial E_2}{\partial Z_4} \frac{\partial Z_4}{\partial A_1} $

$\frac{\partial E_1}{\partial Z_4} = \frac{\partial E_1}{\partial A_4} \frac{\partial A_4}{\partial Z_4}$

In [ ]:
dE2_nach_Z4 = dE_total_nach_A4 * dA4_nach_Z4
dE2_nach_Z4

In [ ]:
dZ4_nach_A1 = w7
dZ4_nach_A1

In [ ]:
dE2_nach_A1 = dE2_nach_Z4 * dZ4_nach_A1
dE2_nach_A1

Insgesamt ergibt sich für $\frac{\partial E_{total}}{\partial A_1} = \frac{\partial E_1}{\partial A_1} +  \frac{\partial E_2}{\partial A_1} $ also:

In [ ]:
dE_total_nach_A1 = dE1_nach_A1 + dE2_nach_A1
dE_total_nach_A1

Konzentrieren wir uns wieder auf die Ausgangsgleichung: $\frac{\partial E_{total}}{\partial w_1} = \frac{\partial E_{total}}{\partial A_1} \frac{\partial A_1}{\partial Z_1} \frac{\partial Z_1}{\partial w_1}   $

Wir benötigen noch $\frac{\partial A_1}{\partial Z_1}$ und $\frac{\partial Z_1}{\partial w_1}$.

$\frac{\partial A_1}{\partial Z_1} = A_1 (1 - A_1)$

In [ ]:
dA1_nach_Z1 = A1 * (1 - A1)
dA1_nach_Z1

$\frac{\partial Z_1}{\partial w_1} = \frac{\partial Z_1}{\partial w_1} (w_1 X_1 + w_2 X_2 + b_1) = X_1$

In [ ]:
dZ1_nach_w1 = X1

Insgesamt ergibt sich für $\frac{\partial E_{total}}{\partial w_1} = \frac{\partial E_{total}}{\partial A_1} \frac{\partial A_1}{\partial Z_1} \frac{\partial Z_1}{\partial w_1}   $:

In [ ]:
dE_total_nach_w1 = dE_total_nach_A1 * dA1_nach_Z1 * dZ1_nach_w1
dE_total_nach_w1

#### Gewicht $w_1$ anpassen

Wir können jetzt das Gewicht $w_1$ mit $w_{1 neu} = w_1 - \alpha \cdot \frac{\partial E_{total}}{\partial w_1}$ anpassen:

In [ ]:
w1_neu = w1 - 0.5 * dE_total_nach_w1
w1_neu

In [ ]:
# Änderung für w1
delta_w1 = w1_neu - w1
delta_w1

### Änderung von $E_{total}$ nach $w_2$ 

<img src="./images/backprop_w2_f.png" alt="drawing" width="80%"/>

$\frac{\partial E_{total}}{\partial w_2} = \frac{\partial E_{total}}{\partial A_1} \frac{\partial A_1}{\partial Z_1} \frac{\partial Z_1}{\partial w_2}   $

$\frac{\partial E_{total}}{\partial A_1} = \frac{\partial E_1}{\partial A_1} +  \frac{\partial E_2}{\partial A_1} $

$\frac{\partial Z_1}{\partial w_2} = \frac{\partial Z_1}{\partial w_2} (w_1 X_1 + w_2 X_2 + b_1) = X_2$

In [ ]:
dZ1_nach_w2 = X2
dZ1_nach_w2

In [ ]:
dE_total_nach_w2 = dE_total_nach_A1 * dA1_nach_Z1 * dZ1_nach_w2
dE_total_nach_w2

#### Gewicht $w_2$ anpassen

Wir können jetzt das Gewicht $w_1$ mit $w_{1 neu} = w_1 - \alpha \cdot \frac{\partial E_{total}}{\partial w_1}$ anpassen:

In [ ]:
w2_neu = w2 - 0.5 * dE_total_nach_w2
w2_neu

In [ ]:
# Änderung für w2
delta_w2 = w2_neu - w2
delta_w2

### Änderung von $E_{total}$ nach $w_3$ 

<img src="./images/backprop_w3_f.png" alt="drawing" width="80%"/>

$\frac{\partial E_{total}}{\partial w_3} = \frac{\partial E_{total}}{\partial A_2} \frac{\partial A_2}{\partial Z_2} \frac{\partial Z_2}{\partial w_3}   $

$\frac{\partial E_{total}}{\partial A_2} = \frac{\partial E_1}{\partial A_2} +  \frac{\partial E_2}{\partial A_2} $

#### 1.)

$\frac{\partial E_1}{\partial A_2} = \frac{\partial E_1}{\partial Z_3} \frac{\partial Z_3}{\partial A_2}$

Wir haben $\frac{\partial E_1}{\partial Z_3}$ bereits ausgerechnet da gilt:

$\frac{\partial E_1}{\partial Z_3} = \frac{\partial E_1}{\partial A_3} \frac{\partial A_3}{\partial Z_3}$

In [ ]:
dE1_nach_Z3 = dE_total_nach_A3 * dA3_nach_Z3
dE1_nach_Z3

$\frac{\partial Z_3}{\partial A_2}$ ergibt sich zu:

$\frac{\partial Z_3}{\partial A_2} = \frac{\partial }{\partial A_2} (w_5 A_1 + w_6 A_2 + b_2) = w_6$

In [ ]:
dZ3_nach_A2 = w6
dZ3_nach_A2

Insgesamt ergibt sich also für $\frac{\partial E_1}{\partial A_2} = \frac{\partial E_1}{\partial Z_3} \frac{\partial Z_3}{\partial A_2}$:

In [ ]:
dE1_nach_A2 = dE1_nach_Z3 * dZ3_nach_A2
dE1_nach_A2

#### 2.)

$\frac{\partial E_2}{\partial A_2} = \frac{\partial E_2}{\partial Z_4} \frac{\partial Z_4}{\partial A_2} $

$\frac{\partial E_2}{\partial Z_4} = \frac{\partial E_2}{\partial A_4} \frac{\partial A_4}{\partial Z_4}$

In [ ]:
dE2_nach_Z4 = dE_total_nach_A4 * dA4_nach_Z4
dE2_nach_Z4

In [ ]:
dZ4_nach_A2 = w8
dZ4_nach_A2

In [ ]:
dE2_nach_A2 = dE2_nach_Z4 * dZ4_nach_A2
dE2_nach_A2

Insgesamt ergibt sich für $\frac{\partial E_{total}}{\partial A_2} = \frac{\partial E_1}{\partial A_2} +  \frac{\partial E_2}{\partial A_2} $ also:

In [ ]:
dE_total_nach_A2 = dE1_nach_A2 + dE2_nach_A2
dE_total_nach_A2

Konzentrieren wir uns wieder auf die Ausgangsgleichung: $\frac{\partial E_{total}}{\partial w_3} = \frac{\partial E_{total}}{\partial A_2} \frac{\partial A_2}{\partial Z_2} \frac{\partial Z_2}{\partial w_3}   $

Wir benötigen noch $\frac{\partial A_2}{\partial Z_2}$ und $\frac{\partial Z_2}{\partial w_3}$.

$\frac{\partial A_2}{\partial Z_2} = A_2 (1 - A_2)$

In [ ]:
dA2_nach_Z2 = A2 * (1 - A2)
dA2_nach_Z2

$\frac{\partial Z_2}{\partial w_3} = \frac{\partial }{\partial w_3} (w_3 X_1 + w_4 X_2 + b_1) = X_1$

In [ ]:
dZ2_nach_w3 = X1
dZ2_nach_w3

Insgesamt ergibt sich für $\frac{\partial E_{total}}{\partial w_3} = \frac{\partial E_{total}}{\partial A_2} \frac{\partial A_2}{\partial Z_2} \frac{\partial Z_2}{\partial w_3}   $:

In [ ]:
dE_total_nach_w3 = dE_total_nach_A2 * dA2_nach_Z2 * dZ2_nach_w3
dE_total_nach_w3

#### Gewicht $w_3$ anpassen

Wir können jetzt das Gewicht $w_3$ mit $w_{3 neu} = w_3 - \alpha \cdot \frac{\partial E_{total}}{\partial w_3}$ anpassen:

In [ ]:
w3_neu = w3 - 0.5 * dE_total_nach_w3
w3_neu

In [ ]:
# Änderung für w3
delta_w3 = w3_neu - w3
delta_w3

### Änderung von $E_{total}$ nach $w_4$ 

<img src="./images/backprop_w4_f.png" alt="drawing" width="80%"/>

$\frac{\partial E_{total}}{\partial w_4} = \frac{\partial E_{total}}{\partial A_2} \frac{\partial A_2}{\partial Z_2} \frac{\partial Z_2}{\partial w_4}   $

$\frac{\partial E_{total}}{\partial A_2} = \frac{\partial E_1}{\partial A_2} +  \frac{\partial E_2}{\partial A_2} $

$\frac{\partial Z_2}{\partial w_4} = \frac{\partial Z_2}{\partial w_4} (w_3 X_1 + w_4 X_2 + b_1) = X_2$

In [ ]:
dZ2_nach_w4 = X2
dZ2_nach_w4

In [ ]:
dE_total_nach_w4 = dE_total_nach_A2 * dA2_nach_Z2 * dZ2_nach_w4
dE_total_nach_w4

Wir können jetzt das Gewicht $w_4$ mit $w_{4 neu} = w_4 - \alpha \cdot \frac{\partial E_{total}}{\partial w_4}$ anpassen:

#### Gewicht $w_4$ anpassen

In [ ]:
w4_neu = w4 - 0.5 * dE_total_nach_w4
w4_neu

In [ ]:
# Änderung für w4
delta_w4 = w4_neu - w4
delta_w4